In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=4a943b8d40be8c68ef4afa7b658db27dceb9c34fabbb83d28ed436a5075083ab
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.keras as ks

from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, mean_poisson_deviance, 
    brier_score_loss, roc_auc_score, roc_curve, RocCurveDisplay
)

import pickle
from datetime import datetime

import wandb
wandb.login(relogin = True)


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Prep Data Once

In [3]:
def filter_nested_array(x, keep):
  result = [[y[key] for key in keep] for y in x]

  return np.array(result, dtype = 'object')

def prepare_row(row):
  # Figure out Claims
  claims_keep_key = ['bi_ind', 'coll_ind', 'comp_ind', 'ers_ind', 'mpc_ind', 'pd_ind', 'ubi_ind',
                      'veh_had_bi_cov_ind', 'veh_had_coll_cov_ind', 'veh_had_comp_cov_ind', 'veh_had_ers_cov_ind', 
                     'veh_had_mpc_cov_ind', 'veh_had_pd_cov_ind', 'veh_had_ubi_cov_ind']
                      
  other_claims = filter_nested_array(row['other_claims'], claims_keep_key)
  other_claim_cnt = len(other_claims)
  if other_claim_cnt > 0:
    other_claims = np.append(other_claims, np.zeros([len(other_claims),1]), 1)

  veh_claims = filter_nested_array(row['vehicle_claims'], claims_keep_key)
  claim_cnt = len(veh_claims)
  if claim_cnt > 0:
    veh_claims = np.append(veh_claims, np.ones([len(veh_claims),1]), 1)

  if claim_cnt + other_claim_cnt == 0:
      all_claims = np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
  elif claim_cnt == 0:
    all_claims = other_claims
  elif other_claim_cnt == 0:
    all_claims = veh_claims
  else:
    all_claims = np.append(veh_claims, other_claims, axis = 0)

  all_claims = tf.ragged.constant([all_claims.astype('float16')], ragged_rank = 1, inner_shape = (15,))

  # Figure Out Drivers
  drivers = filter_nested_array(row['driver_info'], ['driver_age', 'driver_gender', 'driver_tenure'])
  drivers[:, 0] = (drivers[:, 0] - 50)/50
  drivers[:, 1] = np.where(drivers[:, 1] == 'm', 1, 0)
  drivers[:, 2] = (drivers[:, 2] - 10)/10
  drivers = tf.ragged.constant([drivers.astype('float16')], ragged_rank = 1, inner_shape = (3,))

  # Figure out Vehicles
  vehicles = filter_nested_array(row['household_vehicles_info'], ['this_vehicle_ind', 'vehicle_age', 'vehicle_type', 'vehicle_years_owned'])
  vehicles[:, 1] = (vehicles[:, 1] - 15)/15
  vehicles[:, 3] = (vehicles[:, 3] - 15)/15

  veh_type = vehicles[:, 2]
  vehicles[:, 2] = np.where(veh_type == 'van', 1, 0) + np.where(veh_type == 'sedan', 2, 0) + np.where(veh_type == 'sports car', 3, 0) + np.where(veh_type == 'suv', 4, 0)
  vehicles = tf.ragged.constant([vehicles.astype('float16')], ragged_rank = 1, inner_shape = (4,))

  other = [(row['credit_score'] - 600)/500,
           (row['garaging_location'] == 'country') * 1 + (row['garaging_location'] == 'downtown') * 2, 
           (row['household_tenure'] - 15)/10,
           (row['multiline_houses']/2),
           row['multiline_rental'],
           row['multiline_personal_article_policy'],
           row['multiline_personal_liability_umbrella'],
           (row['vehicle_count']-3)/3,
           (row['annual_mileage'] - 10000)/10000,
           (row['vehicle_age'] - 15)/15,
           np.where(row['vehicle_type'] == 'van', 1, 0) + np.where(row['vehicle_type'] == 'sedan', 2, 0) + np.where(row['vehicle_type'] == 'sports car', 3, 0) + np.where(row['vehicle_type'] == 'suv', 4, 0),
           (row['vehicle_years_owned'] - 10)/15
           ]
  other = tf.constant(value = np.array(other, dtype = 'float16'))

  result = {
      'driver_info': drivers,
      'vehicle_info': vehicles,
      'claims_info': all_claims,
      'other_data': other,
      'target': row['vehicle_claim_cnt_pd_0']
  }

  return result

ragged = lambda y : tf.concat(y.to_list(), axis = 0)

def prep_one_datas(features):

  x = [
      ragged(features['driver_info']), 
      ragged(features['vehicle_info']), 
      ragged(features['claims_info']), 
      tf.convert_to_tensor(features['other_data'].to_list())
  ]

  y = tf.convert_to_tensor(features['target'].values)

  return x,y

In [4]:
with wandb.init(
      project="claims_modeling",
      group = 'Data Prep',
      name = f'Data Prep for NN - {datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}',
      notes="Prep for NN Training, No Model Build!",
      tags=["data"]) as run:
    datas = run.use_artifact('msds_498_claims_modeling/claims_modeling/sythetic_data:v5')
    directory = datas.download(root = 'datasets')

    train_features = pd.read_parquet('datasets/split=train') \
      .apply(prepare_row, axis = 1, result_type = 'expand')

    test_features = pd.read_parquet('datasets/split=test')\
      .apply(prepare_row, axis = 1, result_type = 'expand')

    val_features = pd.read_parquet('datasets/split=validation')\
      .apply(prepare_row, axis = 1, result_type = 'expand')

    train_x, train_y = prep_one_datas(train_features)
    test_x, test_y = prep_one_datas(test_features)
    val_x, val_y = prep_one_datas(val_features)

wandb: Currently logged in as: tylerrosacker2022 (msds_498_claims_modeling). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact sythetic_data:v5, 153.76MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:7.0


# Model Train Code

In [5]:
def build_model(run):
  # driver_info
  driver_input = ks.Input(shape = (None, 3), name = 'driver_info')
  driver_dense_1 = tf.keras.layers.Dense(run.config['driver_dense'], 
                                        activation=tf.keras.layers.LeakyReLU(alpha=run.config['relu_leakiness']),
                                        name = "Driver_Info_Dense"
                                        )(driver_input)
  driver_agged = tf.math.reduce_sum(driver_dense_1, 1)

  # vehicle_info
  vehicle_input = ks.Input(shape = (None, 4), name = 'vehicle_info')
  vehicle_dense_1 = tf.keras.layers.Dense(run.config['veh_dense'], 
                                          activation=tf.keras.layers.LeakyReLU(alpha=run.config['relu_leakiness']),
                                        name = "Vehicle_Info_Dense"
                                          )(vehicle_input)
  vehicle_agged = tf.math.reduce_sum(vehicle_dense_1, 1)

  # claims_info
  claims_input = ks.Input(shape = (None, 15), name = 'claims_info')
  claims_dense_1 = tf.keras.layers.Dense(run.config['claim_dense'], 
                                        activation=tf.keras.layers.LeakyReLU(alpha=run.config['relu_leakiness']),
                                        name = "Claim_Info_Dense"
                                        )(claims_input)
  claims_agged = tf.math.reduce_sum(claims_dense_1, 1)

  # other_data
  other_input = ks.Input(shape = (12,), name = 'other_data')

  combined = tf.keras.layers.Concatenate()([driver_agged, vehicle_agged, claims_agged, other_input])
  combined_norm = tf.keras.layers.BatchNormalization()(combined)

  dense_1 = tf.keras.layers.Dense(run.config['dense_res_block_width'], 
                                  activation=tf.keras.layers.LeakyReLU(alpha=run.config['relu_leakiness']),
                                  name = "Res_Layer_1"
                                  )(combined_norm)
  drop_1 = tf.keras.layers.Dropout(run.config['dropout'])(dense_1)

  dense_2 = tf.keras.layers.Dense(run.config['dense_res_block_width'], 
                                  activation=tf.keras.layers.LeakyReLU(alpha=run.config['relu_leakiness']),
                                  name = "Res_Layer_2"
                                  )(drop_1)

  drop_2 = tf.keras.layers.Dropout(run.config['dropout'])(dense_2)

  res_layer = tf.keras.layers.Add(name = "Combined_Res_Result")([drop_1, drop_2])

  target_layer = tf.keras.layers.Dense(1, 
                                      activation=tf.keras.activations.exponential, 
                                      name = 'target')(res_layer)

  model = ks.Model(inputs = [driver_input, vehicle_input, claims_input, other_input],
                outputs = [target_layer])

  model.compile(
                optimizer=tf.keras.optimizers.experimental.AdamW(
                    learning_rate = run.config['learning_rate'],
                    weight_decay = run.config['weight_decay']
                ), 
                loss=tf.keras.losses.Poisson())
  
  return model

In [6]:
def log_stats(dataset_name, prediction, truth):
  prediction = np.clip(prediction, a_min = 0.001, a_max = np.inf)
  predicted_p_gt_0 = np.clip(1 - np.exp(-prediction), a_min = 0, a_max = 1)
  truth_capped = np.clip(truth, a_min = 0, a_max = 1)

  fpr, tpr, _ = roc_curve(truth_capped, predicted_p_gt_0)
  roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()
  
  metrics = {
      f"{dataset_name}_prediction_dist": wandb.Histogram(prediction),
      f"{dataset_name}_mse": mean_squared_error(truth, prediction), 
      f"{dataset_name}_mae": mean_absolute_error(truth, prediction),
      f"{dataset_name}_mean_poisson_deviance": mean_poisson_deviance(truth, prediction),
      f"{dataset_name}_brier_loss": brier_score_loss(truth_capped, predicted_p_gt_0),
      f"{dataset_name}_auc_score": roc_auc_score(truth_capped, predicted_p_gt_0),
      f"{dataset_name}_roc": roc_display.figure_
    }
  wandb.log(metrics)
  


In [7]:
def main(config = None):
  with wandb.init(
      project="claims_modeling",
      group = 'NN Template V1',
      name = f'NN Train - {datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}',
      notes="Architecture 1 NN",
      tags=["nn"],
      save_code = True,
      sync_tensorboard=True,
      config=config) as run:
    datas = run.use_artifact('msds_498_claims_modeling/claims_modeling/sythetic_data:v5')
    model = build_model(run)

    tf.keras.utils.plot_model(
      model,
      to_file='model.png',
      show_shapes=True,
      show_layer_names=True,
      show_layer_activations=True,
      show_trainable=True
    )

    artifact = wandb.Artifact(
        name='model_arch_graph', 
        type='image'
        )    

    artifact.add_file(local_path='model.png')
    run.log_artifact(artifact)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(histogram_freq=1)

    model.fit(train_x, 
              train_y, 
              epochs = run.config['epochs'], 
              batch_size = run.config['batch_size'], 
              validation_data=(test_x, test_y),
              callbacks=[tensorboard_callback])
    
    train_pred = model.predict(train_x, batch_size = 1024)
    test_pred = model.predict(test_x, batch_size = 1024)
    val_pred = model.predict(val_x, batch_size = 1024)

    log_stats('train', train_pred, train_y)
    log_stats('test', test_pred, test_y)
    log_stats('val', val_pred, val_y)

    model.save('model')
    wandb.save('model')

# Grid Search!

In [ ]:
# main(config = {
#         "epochs": 100,
#         "learning_rate": 1e-3,
#         "weight_decay": 4e-3,
#         "relu_leakiness": 0.01,
#         "driver_dense": 4,
#         "veh_dense": 5,
#         "claim_dense": 10,
#         "dense_res_block_width": 25,
#         "dropout": 0.1,
#         "batch_size": 1024
#   })

wandb.agent(entity = "msds_498_claims_modeling", project = "claims_modeling", sweep_id="koeqejgd", function=main, count = 15)

wandb: Agent Starting Run: oyc5fksr with config:
wandb: 	batch_size: 256
wandb: 	claim_dense: 10
wandb: 	dense_res_block_width: 43
wandb: 	driver_dense: 4
wandb: 	dropout: 0.29374955518520646
wandb: 	epochs: 25
wandb: 	learning_rate: 0.0018974163167356484
wandb: 	relu_leakiness: 0.019171223690129184
wandb: 	veh_dense: 3
wandb: 	weight_decay: 1.2496176548696622e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


Epoch 1/25
859/861 [============================>.] - ETA: 0s - loss: 0.3733

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


861/861 [==============================] - 12s 8ms/step - loss: 0.3732 - val_loss: 0.2819
Epoch 2/25
861/861 [==============================] - 8s 8ms/step - loss: 0.2900 - val_loss: 0.2793
Epoch 3/25
861/861 [==============================] - 8s 8ms/step - loss: 0.2833 - val_loss: 0.2794
Epoch 4/25
268/861 [========>.....................] - ETA: 3s - loss: 0.2773